<a href="https://colab.research.google.com/github/lucasmsorrentino/FrameworksAI/blob/main/3_Resolu%C3%A7%C3%A3o_do_Exerc%C3%ADcio_de_Sistemas_de_Recomenda%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3 Resolução do Exercício de Sistemas de Recomendação

---

In [ ]:
# ==========================================
# CÓDIGO FINAL E ROBUSTO (LIVROS)
# ==========================================

# 1. SETUP DO AMBIENTE (LEGACY)
!pip install -q tensorflow-recommenders
import os
# OBRIGATÓRIO: Forçar Keras 2 antes de importar TensorFlow
os.environ['TF_USE_LEGACY_KERAS'] = '1'

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

print(f"Versão TF: {tf.__version__}")

# 2. CARREGAMENTO E CORREÇÃO DOS DADOS
print("\n=== A CARREGAR DADOS ===")

# Procura o ficheiro automaticamente
files = [f for f in os.listdir() if 'Base_livros' in f and not f.endswith('.csv')] # Prioriza Excel
if not files:
    # Se não achar Excel, tenta CSV
    files = [f for f in os.listdir() if 'Base_livros' in f]

if not files:
    raise FileNotFoundError("❌ ERRO: Ficheiro 'Base_livros' não encontrado. Faça upload!")

filename = files[0]
print(f"-> A ler: {filename}")

try:
    # Tenta ler o ficheiro
    if filename.endswith('.xlsx'):
        df = pd.read_excel(filename)
    else:
        df = pd.read_csv(filename, on_bad_lines='skip')

    # --- DIAGNÓSTICO DE COLUNAS ---
    print(f"-> Colunas detetadas inicialmente: {df.columns.tolist()}")

    # --- CORREÇÃO DE FORMATO "AGLOMERADO" ---
    # Verifica se a primeira coluna contém vírgulas no nome (sinal de CSV mal formatado)
    primeira_coluna = df.columns[0]

    # Se 'ID_usuario' NÃO existe, mas a 1ª coluna parece conter os dados misturados...
    if 'ID_usuario' not in df.columns and ',' in str(primeira_coluna):
        print("⚠️ Detetado formato misturado (CSV dentro de Excel). A corrigir à força...")

        # Pega os nomes corretos do cabeçalho "ISBN,Titulo,Autor..."
        novos_nomes = str(primeira_coluna).replace('"', '').split(',')

        # Separa a primeira coluna pelos dados
        df_split = df.iloc[:, 0].astype(str).str.split(',', expand=True)

        # Garante que não temos colunas a mais
        if df_split.shape[1] > len(novos_nomes):
            df_split = df_split.iloc[:, :len(novos_nomes)]

        # Atribui os nomes
        df_split.columns = novos_nomes
        df = df_split
        print("✅ Correção aplicada com sucesso!")

    # Limpeza final de nomes (remove espaços extras)
    df.columns = df.columns.astype(str).str.strip()

    # --- VERIFICAÇÃO FINAL ---
    if 'ID_usuario' not in df.columns:
        print("\n❌ ERRO CRÍTICO: As colunas ainda estão erradas.")
        print(f"O Python está a ler isto: {df.columns.tolist()}")
        print("Tente converter o seu Excel para CSV 'verdadeiro' no Excel (Salvar Como > CSV UTF-8).")
        raise KeyError("Coluna ID_usuario não encontrada")

    # Limpeza dos valores
    df['ID_usuario'] = df['ID_usuario'].astype(str).str.replace('"', '').str.strip()
    df['Titulo'] = df['Titulo'].astype(str).str.replace('"', '').str.strip()

    print("✅ Dados prontos!")

except Exception as e:
    print(f"❌ Ocorreu um erro na leitura: {e}")
    raise

# 3. PREPARAR MODELO
print("\n=== A TREINAR MODELO ===")

data = df[['ID_usuario', 'Titulo']]
ratings_ds = tf.data.Dataset.from_tensor_slices(dict(data))
books_ds = tf.data.Dataset.from_tensor_slices(data['Titulo'].unique())

ratings_ds = ratings_ds.map(lambda x: {"user_id": x["ID_usuario"], "book_title": x["Titulo"]})

# Vocabulários
user_lookup = tf.keras.layers.StringLookup(mask_token=None)
user_lookup.adapt(ratings_ds.map(lambda x: x["user_id"]))

book_lookup = tf.keras.layers.StringLookup(mask_token=None)
book_lookup.adapt(books_ds)

num_users = user_lookup.vocabulary_size()
num_books = book_lookup.vocabulary_size()
print(f"-> {num_users} usuários e {num_books} livros.")

# Classes do Modelo (Customizadas para estabilidade)
class UserTower(tf.keras.Model):
    def __init__(self, lookup, vocab_size, emb_dim=32):
        super().__init__()
        self.lookup = lookup
        self.embedding = tf.keras.layers.Embedding(vocab_size, emb_dim)
    def call(self, inputs):
        return self.embedding(self.lookup(inputs))

class BookTower(tf.keras.Model):
    def __init__(self, lookup, vocab_size, emb_dim=32):
        super().__init__()
        self.lookup = lookup
        self.embedding = tf.keras.layers.Embedding(vocab_size, emb_dim)
    def call(self, inputs):
        return self.embedding(self.lookup(inputs))

class BookRecModel(tfrs.Model):
    def __init__(self):
        super().__init__()
        self.user_model = UserTower(user_lookup, num_users)
        self.book_model = BookTower(book_lookup, num_books)
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=books_ds.batch(128).map(self.book_model)
            )
        )
    def compute_loss(self, features, training=False):
        u = self.user_model(features["user_id"])
        b = self.book_model(features["book_title"])
        return self.task(u, b)

# Treinar
model = BookRecModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
model.fit(ratings_ds.batch(4096).cache(), epochs=3)

# 4. TESTAR
print("\n=== RESULTADO ===")
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
  tf.data.Dataset.zip((books_ds.batch(100), books_ds.batch(100).map(model.book_model)))
)

test_user = '276725'
print(f"Top 3 sugestões para o usuário {test_user}:")
_, titles = index(np.array([test_user]))
for i, title in enumerate(titles[0, :3]):
    print(f"{i+1}: {title.numpy().decode('utf-8')}")

Versão TF: 2.19.0

=== A CARREGAR DADOS ===
-> A ler: Base_livros.xlsx
-> Colunas detetadas inicialmente: ['ISBN,Titulo,Autor,Ano,Editora,ID_usuario,Notas', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']
⚠️ Detetado formato misturado (CSV dentro de Excel). A corrigir à força...
✅ Correção aplicada com sucesso!
✅ Dados prontos!

=== A TREINAR MODELO ===
-> 13857 usuários e 115215 livros.
Epoch 1/3
32/32 [==============================] - 756s 24s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 4.4242e-04 - factorized_top_k/top_10_categorical_accuracy: 6.4422e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0013 - factorized_top_k/top_100_categorical_accuracy: 0.0016 - loss: 32853.9503 - regularization_loss: 0.0000e+00 - total_loss: 32853.9503
Epoch 2/3
32/32 [==============================] - 665s 20s/step - factorized_top_k/top_1_categorical_accuracy: 6.2093e-05 - factorized_top_k/to



---
<br>

1. Desempenho do Modelo O modelo foi treinado com sucesso numa base de dados
considerável (13.857 usuários e 115.215 livros).
    * Convergência: Observamos que a perda (loss) diminuiu progressivamente ao longo das 3 épocas (de 32853 para 31657), o que prova que a rede neural aprendeu padrões de associação entre leitores e livros.
    * Precisão: A métrica top_100_categorical_accuracy subiu para cerca de 3% (0.0300). Embora pareça um número baixo, num universo de 115 mil livros possíveis, o modelo conseguir colocar o livro "correto" entre os 100 primeiros em 3% das vezes (após apenas 3 épocas de treino) demonstra que o sistema de Retrieval (Recuperação) está a funcionar muito melhor que um sorteio aleatório.

    

2. As Recomendações para o Usuário 276725 O sistema sugeriu os seguintes livros:

    1.   Men Are from Mars... (Homens são de Marte...) - Clássico de
    2.   Shattered Trust (Harlequin Presents) - Romance.
    3.   Sudden Fire (Harlequin Presents) - Romance.
<br>

> Interpretação: O modelo identificou um padrão de preferência claro. O sistema agrupou este utilizador no espaço vetorial próximo de livros de romance e relacionamentos (notável pela presença da editora Harlequin, famosa por romances de banca). Isto indica que a "Torre do Usuário" e a "Torre do Livro" conseguiram alinhar corretamente os vetores de gosto literário.